## Training and evaluation of decision transformer

---

> Internship neural networks
>
> Group 4: Reinforcement learning
>
> Deadline 28.02.23 23:59

---

In [1]:
import csv
from datetime import datetime
import numpy as np
import os
import random
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader

In [2]:
%run "../Environment/Connect4.ipynb"
%run "../utils/utils.ipynb"
%run "../OtherAgents/Agents.ipynb"
%run "utils.ipynb"
%run "DecisionTransformer.ipynb"

# Training parameters and evaluation device

The used hyperparameters in this notebook are examples for the code submission. In our paper we present the results for different parameters.

In [3]:
rtg_target = 10            # target return to go (decision transformer looks for trajectories with this rtg)
env_name = 'Connect4'      # name of environment  
vocab_size = 7             # number of actions
state_dim = 42             # dimension of the observations
act_dim = 1                # dimension of the actions
max_timestep=21            # maximum length of a game

num_eval_ep = 100           # num of evaluation episodes per iteration

batch_size = 64            # training batch size
lr = 1e-4                   # learning rate
wt_decay = 1e-4             # weight decay
warmup_steps = 10000        # warmup steps for lr scheduler

# total updates of dt = max_train_iters x num_updates_per_iter
num_updates_per_iter = 1000

context_len = 10        # K in decision transformer
n_blocks = 4            # num of transformer blocks
hidden_dim = 128         # hidden dim of transformer
n_heads = 2             # num of transformer heads
dropout_p = 0.1         # dropout probability

# saves model and csv in this directory
log_dir = "./dt_training/"

if not os.path.exists(log_dir):
    os.makedirs(log_dir)

# evaluation device
device_name = 'cuda'
device = torch.device(device_name)
print("device set to: ", device)

device set to:  cuda


# Create evaluation file + path

In [4]:
start_time = datetime.now().replace(microsecond=0)

start_time_str = start_time.strftime("%y-%m-%d-%H-%M-%S")

prefix = "dt_" + env_name + "_batch_size=" + str(batch_size) + "_context_len=" + str(context_len) + "_n_blocks=" + str(n_blocks) + "_hidden_dim=" + str(hidden_dim) + "_n_heads=" + str(n_heads)

save_model_name =  prefix + "_model_" + start_time_str + ".pt"
save_model_path = os.path.join(log_dir, save_model_name)
save_best_model_path = save_model_path[:-3] + "_best.pt"
save_best_model_path_NM = save_model_path[:-3] + "_best_againstNM.pt"

log_name = prefix + "_log_" + start_time_str + ".csv"
log_path = os.path.join(log_dir, log_name)


csv_writer = csv.writer(open(log_path, 'a', 1))
csv_header = (["duration", "num_updates", "action_loss", 
               "eval_avg_reward", "eval_avg_ep_len", "eval_win_rate",
               "eval_avg_rewardP2", "eval_avg_ep_lenP2", "eval_win_rateP2",
               "eval_avg_rewardNM", "eval_avg_ep_lenNM", "eval_win_rateNM",
               "eval_avg_rewardP2NM", "eval_avg_ep_lenP2NM", "eval_win_rateP2NM"])

csv_writer.writerow(csv_header)


print("=" * 60)
print("start time: " + start_time_str)
print("=" * 60)

print("device set to: " + str(device))
print("model save path: " + save_model_path)
print("log csv save path: " + log_path)

max_avg_reward = -1.0
max_avg_reward_NM = -1.0
total_updates = 0

start time: 23-02-28-17-54-34
device set to: cuda
model save path: ./dt_training/dt_Connect4_batch_size=64_context_len=10_n_blocks=4_hidden_dim=128_n_heads=2_model_23-02-28-17-54-34.pt
log csv save path: ./dt_training/dt_Connect4_batch_size=64_context_len=10_n_blocks=4_hidden_dim=128_n_heads=2_log_23-02-28-17-54-34.csv


# Define agent and optimizer

In [5]:
torch.manual_seed(42)
random.seed(42)

agent = DTAgent(state_dim=state_dim,
            act_dim=act_dim,
            n_blocks=n_blocks,
            hidden_dim=hidden_dim,
            context_len=context_len,
            n_heads=n_heads,
            drop_p=dropout_p,
            rtg_target = rtg_target,
            vocab_size = vocab_size)

optimizer = torch.optim.AdamW(agent.model.parameters(), lr=lr, weight_decay=wt_decay)

scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lambda steps: min((steps+1)/warmup_steps, 1))

env = Connect4()

randomPlayer = RandomAgent()
negaMax = NegaMaxAgent(env, 4)

# Training and evaluation

In [6]:
datasets = [read_list('Dataset/RandomAgentVsRandomAgentDatasetWithoutDups'),
            read_list('Dataset/NegaMaxAgentVsRandomAgentDatasetWithoutDups'), 
            read_list('Dataset/NegaMaxAgentVsNegaMaxAgentDatasetWithoutDups')]

max_train_iters = [20, 30, 30]

# start training
for i in range(len(datasets)):
    # update dataset
    traj_dataset = TrajectoryDataset(datasets[i], context_len)
    
    # Create DataLoader for dataset
    traj_data_loader = DataLoader(traj_dataset,
                            batch_size=batch_size,
                            shuffle=True,
                            pin_memory=True,
                            drop_last=True) 

    data_iter = iter(traj_data_loader)

    for i_train_iter in tqdm(range(max_train_iters[i])):

        log_action_losses = []
        agent.model.train()

        for _ in range(num_updates_per_iter):
            try:
                timesteps, states, actions, returns_to_go, traj_mask = next(data_iter)
            except StopIteration:
                data_iter = iter(traj_data_loader)
                timesteps, states, actions, returns_to_go, traj_mask = next(data_iter)

            timesteps = timesteps.to(device)    # batch_size x traj_length
            states = states.to(device)          # batch_size x traj_length x state_dim
            actions = actions.to(device)        # batch_size x traj_length x act_dim
            returns_to_go = returns_to_go.to(device).unsqueeze(dim=-1) # batch_size x traj_length x 1
            traj_mask = traj_mask.to(device)    # B x T

            _, loss = agent.model.forward(timesteps=timesteps, 
                                                            states=states, 
                                                            actions=actions, 
                                                            returns_to_go=returns_to_go,
                                                            traj_mask = traj_mask)
            
            # optimize loss
            optimizer.zero_grad()
            loss.requires_grad_()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(agent.model.parameters(), 1.0)
            optimizer.step()
            scheduler.step()

            log_action_losses.append(loss.detach().cpu().item())

        # evaluate on env
        results = evaluate_on_env(agent, randomPlayer, "p1", env, num_eval_ep, max_timestep)

        resultsPlayer2 = evaluate_on_env(agent, randomPlayer, "p2", env, num_eval_ep, max_timestep)

        resultsAgainstNM = evaluate_on_env(agent, negaMax, "p1", env, num_eval_ep, max_timestep)

        resultsPlayer2AgainstNM = evaluate_on_env(agent, negaMax, "p2", env, num_eval_ep, max_timestep)

        # get the results from the evaluation
        eval_avg_reward = results['avg_reward']
        eval_avg_ep_len = results['avg_ep_len']
        eval_win_rate = results['win_rate']

        eval_avg_reward_player2 = resultsPlayer2['avg_reward']
        eval_avg_ep_len_player2 = resultsPlayer2['avg_ep_len']
        eval_win_rate_player2 = resultsPlayer2['win_rate']

        eval_avg_reward_AgainstNM = resultsAgainstNM['avg_reward']
        eval_avg_ep_len_AgainstNM = resultsAgainstNM['avg_ep_len']
        eval_win_rate_AgainstNM = resultsAgainstNM['win_rate']

        eval_avg_reward_player2_AgainstNM = resultsPlayer2AgainstNM['avg_reward']
        eval_avg_ep_len_player2_AgainstNM = resultsPlayer2AgainstNM['avg_ep_len']
        eval_win_rate_player2_AgainstNM = resultsPlayer2AgainstNM['win_rate']


        mean_action_loss = np.mean(log_action_losses)
        time_elapsed = str(datetime.now().replace(microsecond=0) - start_time)

        total_updates += num_updates_per_iter
        
        # print results
        log_str = ("=" * 60 + '\n' +
                "time elapsed: " + time_elapsed  + '\n' +
                "num of updates: " + str(total_updates) + '\n' +
                "action loss: " +  format(mean_action_loss, ".5f") + '\n' +
                "eval avg reward: " + format(eval_avg_reward, ".5f") + '\n' +
                "eval avg ep len: " + format(eval_avg_ep_len, ".5f") + '\n' +
                "eval_win_rate: " + format(eval_win_rate, ".2f") + '\n'
                "eval avg reward as player2: " + format(eval_avg_reward_player2, ".5f") + '\n' +
                "eval avg ep len as player2: " + format(eval_avg_ep_len_player2, ".5f") + '\n' +
                "eval_win_rate as player2: " + format(eval_win_rate_player2, ".2f") + '\n' +
                "eval avg reward against NM: " + format(eval_avg_reward_AgainstNM, ".5f") + '\n' +
                "eval avg ep len against NM: " + format(eval_avg_ep_len_AgainstNM, ".5f") + '\n' +
                "eval_win_rate against NM: " + format(eval_win_rate_AgainstNM, ".2f") + '\n' + 
                "eval avg reward as player2 against NM: " + format(eval_avg_reward_player2_AgainstNM, ".5f") + '\n' +
                "eval avg ep len as player2 against NM: " + format(eval_avg_ep_len_player2_AgainstNM, ".5f") + '\n' +
                "eval_win_rate as player2 against NM: " + format(eval_win_rate_player2_AgainstNM, ".2f"))

        print(log_str)
        
        # save results in csv
        log_data = [time_elapsed, total_updates, mean_action_loss,
                    eval_avg_reward, eval_avg_ep_len, 
                    eval_win_rate, eval_avg_reward_player2,
                    eval_avg_ep_len_player2, 
                    eval_win_rate_player2,
                    eval_avg_reward_AgainstNM, eval_avg_ep_len_AgainstNM,
                    eval_win_rate_AgainstNM,
                    eval_avg_reward_player2_AgainstNM, eval_avg_ep_len_player2_AgainstNM,
                    eval_win_rate_player2_AgainstNM]

        csv_writer.writerow(log_data)

        # save best model against random
        print("max avg reward: " + format(max_avg_reward, ".5f"))
        if eval_avg_reward >= max_avg_reward:
            print("saving max avg reward model at: " + save_best_model_path)
            torch.save(agent.model.state_dict(), save_best_model_path)
            max_avg_reward = eval_avg_reward
        # save best model against negaMax
        if eval_avg_reward_AgainstNM >= max_avg_reward_NM:
            print("saving max avg reward model at: " + save_best_model_path_NM)
            torch.save(agent.model.state_dict(), save_best_model_path_NM)
            max_avg_reward_NM = eval_avg_reward_AgainstNM

    
    currentModelPath = save_model_path[:-3] + "_" + str(total_updates) + ".pt"
    print("saving current model at: " + currentModelPath)
    torch.save(agent.model.state_dict(), currentModelPath)


final_model_path = save_model_path      
torch.save(agent.model.state_dict(), final_model_path)
                                             
print("=" * 60)
print("finished training!")
print("=" * 60)
end_time = datetime.now().replace(microsecond=0)
time_elapsed = str(end_time - start_time)
end_time_str = end_time.strftime("%y-%m-%d-%H-%M-%S")
print("started training at: " + start_time_str)
print("finished training at: " + end_time_str)
print("total training time: " + time_elapsed)
print("max avg reward: " + format(max_avg_reward, ".5f"))
print("saved max avg reward model at: " + save_best_model_path)
print("saved last updated model at: " + save_model_path)
print("=" * 60)

csv_writer = open(log_path, 'a', 1).close()

  5%|██                                        | 1/20 [01:54<36:14, 114.46s/it]

time elapsed: 0:02:03
num of updates: 1000
action loss: 1.98899
eval avg reward: 4.20000
eval avg ep len: 8.32000
eval_win_rate: 0.71
eval avg reward as player2: 4.60000
eval avg ep len as player2: 8.97000
eval_win_rate as player2: 0.73
eval avg reward against NM: -10.00000
eval avg ep len against NM: 10.52000
eval_win_rate against NM: 0.00
eval avg reward as player2 against NM: -8.20000
eval avg ep len as player2 against NM: 8.87000
eval_win_rate as player2 against NM: 0.09
max avg reward: -1.00000
saving max avg reward model at: ./dt_training/dt_Connect4_batch_size=64_context_len=10_n_blocks=4_hidden_dim=128_n_heads=2_model_23-02-28-17-54-34_best.pt


 10%|████▏                                     | 2/20 [03:34<31:43, 105.73s/it]

time elapsed: 0:03:42
num of updates: 2000
action loss: 1.95200
eval avg reward: 8.00000
eval avg ep len: 7.38000
eval_win_rate: 0.90
eval avg reward as player2: 7.40000
eval avg ep len as player2: 6.23000
eval_win_rate as player2: 0.87
eval avg reward against NM: -4.80000
eval avg ep len against NM: 9.97000
eval_win_rate against NM: 0.26
eval avg reward as player2 against NM: -9.40000
eval avg ep len as player2 against NM: 6.83000
eval_win_rate as player2 against NM: 0.03
max avg reward: 4.20000
saving max avg reward model at: ./dt_training/dt_Connect4_batch_size=64_context_len=10_n_blocks=4_hidden_dim=128_n_heads=2_model_23-02-28-17-54-34_best.pt


 15%|██████▎                                   | 3/20 [05:21<30:11, 106.54s/it]

...

time elapsed: 3:14:47
num of updates: 78000
action loss: 0.75151
eval avg reward: 8.40000
eval avg ep len: 8.81000
eval_win_rate: 0.92
eval avg reward as player2: 8.20000
eval avg ep len as player2: 8.52000
eval_win_rate as player2: 0.91
eval avg reward against NM: 0.00000
eval avg ep len against NM: 13.64000
eval_win_rate against NM: 0.50
eval avg reward as player2 against NM: 0.60000
eval avg ep len as player2 against NM: 16.77000
eval_win_rate as player2 against NM: 0.57
max avg reward: 9.80000


 97%|█████████████████████████████████████▋ | 29/30 [1:36:33<03:47, 227.01s/it]

time elapsed: 3:18:30
num of updates: 79000
action loss: 0.74519
eval avg reward: 9.40000
eval avg ep len: 8.46000
eval_win_rate: 0.97
eval avg reward as player2: 8.80000
eval avg ep len as player2: 7.95000
eval_win_rate as player2: 0.94
eval avg reward against NM: 2.60000
eval avg ep len against NM: 13.18000
eval_win_rate against NM: 0.63
eval avg reward as player2 against NM: 0.00000
eval avg ep len as player2 against NM: 16.42000
eval_win_rate as player2 against NM: 0.53
max avg reward: 9.80000


100%|███████████████████████████████████████| 30/30 [1:40:23<00:00, 200.79s/it]

time elapsed: 3:22:20
num of updates: 80000
action loss: 0.74033
eval avg reward: 8.60000
eval avg ep len: 8.75000
eval_win_rate: 0.93
eval avg reward as player2: 7.80000
eval avg ep len as player2: 8.47000
eval_win_rate as player2: 0.89
eval avg reward against NM: 5.20000
eval avg ep len against NM: 11.82000
eval_win_rate against NM: 0.76
eval avg reward as player2 against NM: -1.30000
eval avg ep len as player2 against NM: 16.24000
eval_win_rate as player2 against NM: 0.47
max avg reward: 9.80000
saving max avg reward model at: ./dt_training/dt_Connect4_batch_size=64_context_len=10_n_blocks=4_hidden_dim=128_n_heads=2_model_23-02-28-17-54-34_best_againstNM.pt
saving current model at: ./dt_training/dt_Connect4_batch_size=64_context_len=10_n_blocks=4_hidden_dim=128_n_heads=2_model_23-02-28-17-54-34_80000.pt
finished training!
started training at: 23-02-28-17-54-34
finished training at: 23-02-28-21-16-54
total training time: 3:22:20
max avg reward: 9.80000
saved max avg reward model at: 